In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install langgraph
!pip install pyngrok
!pip install flask
!pip install request

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("your auth_token ")
pub_url=ngrok.connect(5000).public_url

In [ ]:
from pyngrok import ngrok

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

import locale
locale.getpreferredencoding = lambda: "UTF-8"

from flask import Flask, request, jsonify
from pyngrok import ngrok


from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

flask_app = Flask(__name__)
model_name='NousResearch/Llama-2-7b-chat-hf'
device_map = {"": 0}
# Reload model in FP16 and merge it with LoRA weights
new_model_local_url="Llama-2-7b-chat-finetune"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model_local_url)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


#///////////////////////////////////////////////////////////////////////////////////////////////



his=''
def_assistant=" <<SYS>> You are a good movie recommendation assistant. <</SYS>> "
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,  max_new_tokens=50)

# Define the function that calls the model
def call_model(state: MessagesState):
    global his
    print("query:",state["messages"][-1].content)
    prompt = his + " User: " + state["messages"][-1].content +". "
    his    = his + " User: " + state["messages"][-1].content +". "

    # prompt=state["messages"][-1].content
    # his+= state["messages"][-1].content
    input_seq= def_assistant + prompt
    result = pipe(f"<s>[INST] {input_seq} [/INST]")

    print("+++++++++++++++++++++++> ")
    print(f"<s>[INST] {input_seq} [/INST]")
    print("+++++++++++++++++++++++> ")


    response = result[0]['generated_text']
    response_1 = response.split('[/INST]')[-1]  # Take text after '[INST]'
    his+="Assistant :" + response_1+". "
    print("/n-----------------------------")
    print(his)
    print("/n--------------------------------")

    return {"messages": response_1}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}


#///////////////////////////////////////////////////////////////////////////////////////////////

print(pub_url)
@flask_app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    input_text = data.get("text", "")
    print("+=============================>",input_text)
    input_messages = [HumanMessage(input_text)]
    output = app.invoke({"messages": input_messages}, config)
    print("+=============================>",output["messages"][-1])
    print("+=============================>",output["messages"][-1].content)

    return jsonify({"response": output["messages"][-1].content})
# tunnel ngrok
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

flask_app.run(port=5000)





In [ ]:
#Empty VRAM
del base_model
del model
del pipe
import gc
gc.collect()
gc.collect()

0